In [1]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2
import logging

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.addHandler(logging.FileHandler('Logs/JTH_reg_{0}.log'.format(datetime.datetime.now().\
                                                            strftime("%Y-%m-%d"), 'a')))
print = logging.info
print('good day to you madam fiona')
print('started')
print(datetime.datetime.now())

INFO:root:good day to you madam fiona
INFO:root:started
INFO:root:2018-07-28 20:44:35.223189


In [3]:
r_f = "DataStore/2018-07-P2/Reg0726/reg_model_0726.pkl"
regs = pickle.load(open(r_f, "rb"))

# Similarity regressions
cit_reg = []
for c in ['perc_match_10','sim_ldavecs_cite_msa_match_10','sim_docvecs_cite_msa_match_10']:
    cit_reg.extend([
        "{0} ~ C(inv_msa_match) + C(tp_gyear)".format(c),
        "{0} ~ C(inv_msa_match) + C(tp_primclass)  + C(tp_gyear)".format(c),
        "norm_{0} ~ C(inv_msa_match) + C(tp_gyear)".format(c),
        "norm_{0} ~ C(inv_msa_match) + C(tp_primclass)  + C(tp_gyear)".format(c),
    ])
    regs["JTH"] = pd.Series(cit_reg)
    
    regs["JTH model names"] = pd.Series([
        "Perc Match Targ MSA, Year FE",
        "Perc Match Targ MSA, PC FE",
        "N Perc Match Targ MSA, Year FE",
        "N Perc Match Targ MSA, PC FE",
        "Sim LDAVecs Match Targ MSA, Year FE",
        "Sim LDAVecs Match Targ MSA, PC FE",
        "N Sim LDAVecs Match Targ MSA, Year FE",
        "N Sim LDAVecs Match Targ MSA, PC FE",
        "Sim DocVecs Match Targ MSA, Year FE",
        "Sim DocVecs Match Targ MSA, PC FE", 
        "N Sim DocVecs Match Targ MSA, Year FE",
        "N Sim DocVecs Match Targ MSA, PC FE", 
    ])
    regs["JTH norm"] = regs["JTH"].loc[[2,3,6,7,10,11]]
pickle.dump(regs, open(r_f, "wb"))

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


In [12]:
info_dict = {'$N$':lambda x: "{0:d}".format(int(x.nobs)),
'Adjusted $R^2$':lambda x: "{:.2f}".format(x.rsquared_adj)}
def get_fit(formula, grouped_data, group_col, cov_type, return_fit = False):
    summ = []
    tables = {}
    
    # Remove missing values used in formula
    col_used = re.findall('\((.*?)\)',formula)
    grouped_data = grouped_data.dropna(how="any", subset=col_used) # Should take care of gyear > 1980 issues
    
    # Group and then get results
    grouped_data = grouped_data.groupby(group_col)
    
    for n,g in grouped_data:
        fit = smf.ols(formula = formula, data = g, missing="drop").fit(cov_type=cov_type)
        # Get results tables
        tables[n] = fit.summary2().tables
        # Append results
        summ.append(fit)
    # Get full results output
    # Dataframe of full results
    res_no_stars = summary2.summary_col(summ, stars = False, \
    model_names = ["{0}".format(n) for n in grouped_data.groups.keys()],\
        info_dict = info_dict).tables[0]
    res_stars = summary2.summary_col(summ, stars = True, \
    model_names = ["{0}".format(n) for n in grouped_data.groups.keys()],\
        info_dict = info_dict).tables[0]
    
    # Get partial results
    # 1. Get relevant variables from index of full results
    regressors = [v for v in res_no_stars.index.unique() if ("sim_" in v) | ("match" in v)]
    # 2. Make sure regressors come last
    regressors = regressors+["Intercept"]
    # 3. Get results with regressors
    part_res_no_stars = summary2.summary_col(summ, stars = False, \
    model_names = ["{0}".format(n) for n in grouped_data.groups.keys()],\
        info_dict = info_dict, regressor_order = regressors).tables[0]
    part_res_stars = summary2.summary_col(summ, stars = True, \
    model_names = ["{0}".format(n) for n in grouped_data.groups.keys()],\
        info_dict = info_dict, regressor_order = regressors).tables[0]
    
    # 4. Get index of where Intercept is and add 2 (to include standard error)
    last_ind = list(part_res_stars.index).index("Intercept")+2
    
    # 5. Get partial results
    part_res_no_stars = pd.concat([part_res_no_stars.iloc[:last_ind], part_res_no_stars.iloc[-2::]])
    part_res_stars = pd.concat([part_res_stars.iloc[:last_ind], part_res_stars.iloc[-2::]])
    
    if return_fit == True:
        return summ, tables, res_no_stars, res_stars, part_res_no_stars, part_res_stars
    else:
        return tables, res_no_stars, res_stars, part_res_no_stars, part_res_stars

In [14]:
print("started")
print(datetime.datetime.now())
# 1. Load file
pathdir = "DataStore/2018-07-P3/JTHReg0727/"
rs = pd.read_pickle(pathdir+"targ_cite_sim_reg_0727.pkl")
regs = pd.read_pickle(open("DataStore/2018-07-P2/Reg0726/reg_model_0726.pkl", "rb"))
print(len(rs))
# 3. Define output
samp_out = {}
formulas = list(regs["JTH"])
formulas_ind = list(regs["JTH"].index)
cov = "HC1"
for i, j in zip(formulas_ind, formulas):
    print(i)
    print(j)
    print(datetime.datetime.now())
    try:
        out = get_fit(j, rs, "year_group", cov, return_fit = False)
        samp_out[i] = {}
        samp_out[i]["model"] = j
        samp_out[i]["tables"] = out[0]
        samp_out[i]["res_no_stars"] = out[1]
        samp_out[i]["res_stars"] = out[2]
        samp_out[i]["part_res_no_stars"] = out[3]
        samp_out[i]["part_res_stars"] = out[4]
    except Exception as e:
        logging.exception("error here")
        pass
    print("finished")
    print(datetime.datetime.now())
pickle.dump(samp_out, open(pathdir+"JTH_cite_res_0727.pkl", "wb"))    

INFO:root:started
INFO:root:2018-07-28 12:29:00.454724
INFO:root:564158
INFO:root:0
INFO:root:perc_match_10 ~ C(inv_msa_match) + C(tp_gyear)
INFO:root:2018-07-28 12:29:00.531908
INFO:root:finished
INFO:root:2018-07-28 12:29:04.113536
INFO:root:1
INFO:root:perc_match_10 ~ C(inv_msa_match) + C(tp_primclass)  + C(tp_gyear)
INFO:root:2018-07-28 12:29:04.115003
INFO:root:finished
INFO:root:2018-07-28 12:29:39.308190
INFO:root:2
INFO:root:norm_perc_match_10 ~ C(inv_msa_match) + C(tp_gyear)
INFO:root:2018-07-28 12:29:39.309743
INFO:root:finished
INFO:root:2018-07-28 12:29:43.051090
INFO:root:3
INFO:root:norm_perc_match_10 ~ C(inv_msa_match) + C(tp_primclass)  + C(tp_gyear)
INFO:root:2018-07-28 12:29:43.052602
INFO:root:finished
INFO:root:2018-07-28 12:30:19.389706
INFO:root:4
INFO:root:sim_ldavecs_cite_msa_match_10 ~ C(inv_msa_match) + C(tp_gyear)
INFO:root:2018-07-28 12:30:19.390859
INFO:root:finished
INFO:root:2018-07-28 12:30:23.065702
INFO:root:5
INFO:root:sim_ldavecs_cite_msa_match_10 ~ 